### Answering Business Questions Using SQL with Chinook data

This project revolves around answering business cases with chinook data. 

Chinook data is a combination of different play stores involving artist play track and album.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:

%%sql

--let me view what my table is made up using the view table code with this code

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


I have 11 different tables in my database. The business questions will revolve arond this table

In [3]:
%%sql 

--always use this to inform jupyter that you are working with sql

SELECT *
FROM customer --select all the columns that you see in customer table and return 3 rows
LIMIT 3;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


In [4]:
%%sql --let us check out for the employee table

SELECT *
FROM employee
LIMIT 3;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


In [5]:
%%sql 

--since we are moving in to the business questions, let us view the invoice table

SELECT * 
FROM invoice
limit 3;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98


In [6]:
%%sql -- let me refresh myself on what the database looks like again

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Selecting Album to purchase.

Description:

The chinook record store has just signed a deal with a new record label, and i have been tasked with selecting the first three(3) albums to be added from the store from a list of four(4).

Artist Name	             Genre

Regal	                 Hip-Hop
Red Tone	             Punk
Meteor and the Girls	 Pop
Slim Jim Bites	         Blues


The record label specializes in artists from the USA and some money for advertisment have been given to chinook to advertise the new albums in the USA.

Business Case:

which genres sells the best in the USA?

so far,

we will be working with the genre table, invoice table, customer table, invoice line, and track table

from the schema diagram, 
the invoice is related with the customer table with a primary key of customer_id.

the invoice table is related to the invoice line tab;le with the key, invoice_id.

the invoice line is related to track with the key track_id

and the track is related to the genre with the key genre_id

We are interested in:

country which is USA (customer table)

genre name(genre table)

number of tracks sold(track_id from invoice line)




In [7]:
%%sql 

WITH country_USA AS --getting the country in USA.
   (
       SELECT
            il.*,
            c.country
        FROM invoice_line AS il
        INNER JOIN invoice AS i ON il.invoice_id = i.invoice_id
        INNER JOIN customer AS c ON i.customer_id = c.customer_id
        WHERE country = "USA"
    
   ),
    
number_of_track AS
    (
        SELECT 
            cu.track_id, 
            t.genre_id,
            cu.invoice_line_id
       FROM country_USA AS cu
       INNER JOIN track AS t ON cu.track_id = t.track_id
    )
    
SELECT 
    g.name AS genre_sold,
    COUNT(nt.invoice_line_id) AS Track_sold,
    CAST(COUNT(nt.invoice_line_id) AS FLOAT) / (SELECT COUNT(*) FROM number_of_track) AS percentage_sold
FROM number_of_track AS nt
INNER JOIN genre AS g on nt.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre_sold,Track_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Generally, it could be seen that most sold genre based on track sold is the Rock which accounts for 561 track sold in the USA which is about 53% of total genre in the USA market. But that is not the business case given out. The business case is to based on the existing market, which three albums should be promoted or added to the store? which include;

Artist Name	             Genre

Regal	                 Hip-Hop,
Red Tone	             Punk,
Meteor and the Girls	 Pop,
Slim Jim Bites	         Blues

we could see that Regal (Hip-Hop is about 1.9% with 2- sold tracks)

Red Tone(Punk is about 12% with 130 track sold)

Meteor and the girls(pop is about 2% with 22 track sold)

Slim Jim Bites(Blues is about 3.4% with about 36 tracks sold)

hence i would recommend in order

Red Tone(Punk)
Slim Jim Bites(Blues)
Meteor and the Girls(Pop)

and i will suggest investing more on Rock which recorded 561 tracks sol and at about 53%

### Analyzing Employee Sales performance

This business analysis involves showing sales agent performance which is as well known as analyzing employee sales performance.

This will be done based on analysis carried out on customer purchase in the company which is automatically directed or attained by each saes agent.

The end product of the project is to see if the any employee is performing better or worse than the others.

This performance has to be worked based on three tables from the database

Employee table having link to customer table with support_rep_id

customer table having link to the invoice table with the invoice_id and 

invoice table containing the total amount purchased by each customer id

Now lets start by looking at each tables.

In [8]:
%%sql --telling jupyter that i am going to work with sql engine

SELECT * 
FROM employee
ORDER BY hire_date DESC
LIMIT 3

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


In [9]:
%%sql --lets move to our customer table

SELECT *
FROM customer
LIMIT 3

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


In [10]:
%%sql --lets move into our invoice table

SELECT * 
FROM invoice 
LIMIT 3

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98


In [11]:
%%sql

WITH total_amount_customer AS --combining our customer and invoice table to extract our total purchase by each customer. 
     (
        SELECT
            i.invoice_id,
            c.customer_id,
            c.support_rep_id,
            SUM(i.total) AS total_purchased
         FROM customer AS c
         INNER JOIN invoice AS i ON c.customer_id = i.customer_id
         GROUP BY 2
         ORDER BY 3 DESC
         
      )
    
SELECT 
    e.first_name||" "||e.last_name AS employee_name,
    e.title AS employee_position,
    e.employee_id,
    e.hire_date AS start_date,
    tac.customer_id,
    SUM(tac.total_purchased) AS employee_success_rate
FROM total_amount_customer AS tac
INNER JOIN employee AS e ON e.employee_id = tac.support_rep_id
GROUP BY 1
ORDER BY 3,5

Done.


employee_name,employee_position,employee_id,start_date,customer_id,employee_success_rate
Jane Peacock,Sales Support Agent,3,2017-04-01 00:00:00,59,1731.5099999999998
Margaret Park,Sales Support Agent,4,2017-05-03 00:00:00,56,1584.0000000000002
Steve Johnson,Sales Support Agent,5,2017-10-17 00:00:00,57,1393.92


We could see that Jean Peacock who is a sales agent recorded the highest success rate by making about 1731.51 total purchase by customers.

while Steve Johnson recorded the least with about 1393.92. Well, there is no much difference between him and the highest which is Jane, but one can attribute the difference from the hiring date. Jane was hired on the month of April 2017 while Steve was hired on the month of October 2017. This is about 5 months difference with about 150 days difference.

Looking back at our employee table, and ordering it to hire_date, we could see that Steve was the latest new bie in the organization. He should be the best performing sales agent with such record rate. 

### Analyzing Sales by Country

In this analysis, i will be interested in the sales data for customers from each country.

My query will collate the data on purchase from different countries.

Since several countries had one purchase, i will group them together in a row named "other".

I will calculate the total number of customers (customer_id from the customer table)

The total value of sales which is the (sum total of the unit_price from the invoice table)

average value of sales per customer(which will be sum of the unit_price divided by the count distinct of the customer_id)

average value order from the invoice_line (which wil be sum of the Unit_pirce divided by the count of the distinct incoice_id)

i will be working with the following table:

customer,

invoice

invoice_line

In [12]:
%%sql --let us begin by bringing out the countries with most sale

SELECT --getting a view on how our country will be grouped with regards to total purchases
    c.country AS country,
    COUNT(DISTINCT c.customer_id) AS customer_id,
    i.invoice_id,
    SUM(i.total) AS total_purchased
FROM customer AS c
INNER JOIN invoice AS i ON c.customer_id = i.customer_id
GROUP BY country
ORDER BY total_purchased DESC



Done.


country,customer_id,invoice_id,total_purchased
USA,13,613,1040.4899999999998
Canada,8,612,535.5900000000001
Brazil,5,597,427.68000000000006
France,5,596,389.0699999999999
Germany,4,592,334.62
Czech Republic,2,588,273.24000000000007
United Kingdom,3,614,245.52
Portugal,2,605,185.13000000000002
India,2,607,183.14999999999998
Ireland,1,604,114.83999999999997


In [13]:

%%sql --let us now build on our business case, not leaving out the countries with 1 customer_id as others


WITH purchase_from_countries AS
    (
        SELECT 
           CASE
                WHEN (
                        SELECT
                            COUNT(*)
                     FROM customer
                     WHERE country = c.country) > 1 THEN c.country
                     ELSE "other"
        END AS country,
        c.customer_id,
        i.invoice_id,
        i.total
        FROM customer AS c
        INNER JOIN invoice AS i ON c.customer_id = i.customer_id
        )
SELECT 
    country,
    total_customers,
    total_values_sales,
    average_value_sales,
    average_value_order
FROM
    (SELECT
         pc.country AS country,
         COUNT(DISTINCT pc.customer_id) AS total_customers, 
         ROUND(SUM(pc.total), 2)AS total_values_sales, --rounding to 2 decimal places
         ROUND(SUM(pc.total) / COUNT(DISTINCT pc.customer_id),2) AS average_value_sales, --rounding to 2 decimal places
         ROUND(SUM(pc.total) / COUNT(DISTINCT pc.invoice_id),2) AS average_value_order,
     
     CASE
         WHEN 
             pc.country = "other" THEN 1
         ElSE 0
         END AS sort
     FROM purchase_from_countries AS pc
     GROUP BY country
     ORDER BY sort ASC, total_values_sales DESC 
     
    )


Done.


country,total_customers,total_values_sales,average_value_sales,average_value_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
other,15,1094.94,73.0,7.45


Based on the result, we could see the three top tiers

USA with total customers of 13 and total sales of 1040

CANADA with total customers of 8 and total sales of 535

BRAZIL with total customers of 5 and total sales of 427

we can deep drive our advert here since the market is already existing at this places.

nevertheless, we should't overlook the concept of new or promising market which exist in countries like

Czech Republic with total customers of 2 and sales of 273
United Kingdom with total customers of 3 and sales of 245
India with the total customers of 2 and sales of 183

i didn't chose portugal since it is the least based on the average value order. but we should't put all our spending here.

In [14]:

%%sql --let us concentrate only on 

WITH purchase_from_countries AS
    (
        SELECT 
           CASE
                WHEN (
                        SELECT
                            COUNT(*)
                     FROM customer
                     WHERE country = c.country) = 1  THEN c.country
                     ELSE "other"
        END AS country,
        c.customer_id,
        i.invoice_id,
        i.total
        FROM customer AS c
        INNER JOIN invoice AS i ON c.customer_id = i.customer_id
        )
SELECT 
    country,
    total_customers,
    total_values_sales,
    average_value_sales,
    average_value_order
FROM
    (SELECT
         pc.country AS country,
         COUNT(DISTINCT pc.customer_id) AS total_customers, 
         ROUND(SUM(pc.total), 2)AS total_values_sales, --rounding to 2 decimal places
         ROUND(SUM(pc.total) / COUNT(DISTINCT pc.customer_id),2) AS average_value_sales, --rounding to 2 decimal places
         ROUND(SUM(pc.total) / COUNT(DISTINCT pc.invoice_id),2) AS average_value_order,
     
     CASE
         WHEN 
             pc.country = pc.country THEN 1
         ElSE 0
         END AS sort
     FROM purchase_from_countries AS pc
     GROUP BY country
     ORDER BY sort ASC, total_values_sales DESC 
     
    )


Done.


country,total_customers,total_values_sales,average_value_sales,average_value_order
other,44,3614.49,82.15,7.74
Ireland,1,114.84,114.84,8.83
Spain,1,98.01,98.01,8.91
Chile,1,97.02,97.02,7.46
Australia,1,81.18,81.18,8.12
Finland,1,79.2,79.2,7.2
Hungary,1,78.21,78.21,7.82
Poland,1,76.23,76.23,7.62
Sweden,1,75.24,75.24,7.52
Norway,1,72.27,72.27,8.03


Looking only at countries with one customer, we can't make a conclusion of channeling our advert to this areas due to limited number but we should watch out for countries like IRELAND with 1 customer and total sales of 173

### Analyzing  Album vs Individual tracks

In [15]:
%%sql

WITH invoice_track AS
    (
     SELECT
         il.invoice_id AS invoice_id,
         MAX(il.track_id) AS max_track_id
     FROM invoice_line AS il
     GROUP BY 1
    )

SELECT
    album_to_buy,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) AS percent
FROM
    (
    SELECT
        it.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2 --recursive join
                                      WHERE t2.track_id = it.max_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = it.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = it.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = it.max_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_to_buy"
     FROM invoice_track it
    )
GROUP BY album_to_buy;

Done.


album_to_buy,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

### Most played artist

In [57]:
%%sql

SELECT
    ar.name,
    SUM(DISTINCT pt.playlist_id) AS num_played,
    a.artist_id,
    a.title
FROM playlist_track AS pt
INNER JOIN track AS t ON pt.track_id = t.track_id
INNER JOIN album AS a ON t.album_id = a.album_id
INNER JOIN artist AS ar ON a.artist_id = ar.artist_id
GROUP BY a.artist_id
ORDER BY num_played DESC

Done.


name,num_played,artist_id,title
Eugene Ormandy,68,226,"Holst: The Planets, Op. 32 & Vaughan Williams: Fantasies"
English Concert & Trevor Pinnock,55,208,Pachelbel: Canon & Gigue
Academy of St. Martin in the Fields & Sir Neville Marriner,55,214,The World of Classical Favourites
The King's Singers,53,247,English Renaissance
Berliner Philharmoniker & Herbert Von Karajan,53,248,Mozart: Symphonies Nos. 40 & 41
Alberto Turco & Nova Schola Gregoriana,41,206,Adorate Deum: Gregorian Chant from the Proper of the Mass
"Richard Marlow & The Choir of Trinity College, Cambridge",41,207,Allegri: Miserere
Wilhelm Kempff,41,211,Bach: Goldberg Variations
Yo-Yo Ma,41,212,Bach: The Cello Suites
Scholars Baroque Ensemble,41,213,Handel: The Messiah (Highlights)


We could see that the most played artist in our playlist is Eugene Ormandy at about 68 and the most played song title is "Holst: The Planets, Op. 32 & Vaughan Williams: Fantasies"

### Purchased and Non-purchased tracks

In [75]:
%%sql -- we are going to look through our store to see purchased vs non purchased tracks

WITH track_purchased AS
    (
        SELECT 
            il.track_id
        
        FROM invoice AS i --to link it up with the invoice_id, we will join it to the invoive table
        INNER JOIN invoice_line AS il ON il.invoice_id = i.invoice_id 
        GROUP BY il.track_id
    )
    
SELECT
    (SELECT COUNT(track_id) FROM track) AS total_track, 0--getting the total tracks in the store
    COUNT(*) track_purchased,
    (SELECT COUNT(track_id) FROM track)  - COUNT(*) AS not_purchased_track

    
FROM track_purchased
INNER JOIN track AS t ON t.track_id = track_purchased.track_id



Done.


total_track,track_purchased,not_purchased_track
3503,1806,1697


### Tracks sold vs available sold tracks

In [101]:
%%sql

WITH track_sold AS --getting the values for our sold tracks in the store
    (
        SELECT
            SUM(i.total) AS total_sales,
            COUNT(il.quantity) AS track_sold,
            g.name AS genre,
            CAST(COUNT(il.quantity) AS FLOAT) / (SELECT COUNT(*)  FROM invoice_line) AS percentage_sold
        FROM invoice AS i
        INNER JOIN invoice_line As il ON i.invoice_id = il.invoice_id
        INNER JOIN track AS t ON t.track_id = il.track_id
        INNER JOIN genre AS g ON g.genre_id = t.genre_id
        GROUP BY 3
        ORDER BY 4
    ),
    
    track_available AS --getting the available tracks in the store
        (
            SELECT
                COUNT(t.track_id) AS available_track,
                g.name AS genre,
                CAST(COUNT(t.track_id) AS FLOAT) / (SELECT COUNT(*) FROM track) AS percentage_available
            FROM track AS t
            INNER JOIN genre AS g ON g.genre_id = t.genre_id
            GROUP BY 2
        )

SELECT --Using left join to ensure that tracks available but not sold will reflect in our store
    ts.genre,
    ts.track_sold,
    ts.total_sales,
    ts.percentage_sold,
    ta.genre,
    ta.available_track, 
    ta.percentage_available
FROM track_available AS ta 
LEFT JOIN track_sold AS ts ON ts.genre = ta.genre
ORDER BY 3 DESC
    



Done.


genre,track_sold,total_sales,percentage_sold,genre_1,available_track,percentage_available
Rock,2635,26751.779999999926,0.553920538154299,Rock,1297,0.3702540679417642
Metal,619,5316.300000000009,0.13012402774858103,Metal,374,0.10676562946046246
Alternative & Punk,492,4841.0999999999985,0.10342652932520496,Alternative & Punk,332,0.09477590636597202
R&B/Soul,159,1751.3100000000056,0.033424427159974776,R&B/Soul,61,0.01741364544675992
Latin,167,1705.7700000000018,0.03510615934412445,Latin,579,0.1652868969454753
Blues,124,1379.069999999998,0.026066848854319948,Blues,81,0.02312303739651727
Jazz,121,1302.84,0.025436199285263823,Jazz,130,0.03711104767342278
Alternative,117,1095.9299999999996,0.024595333193188986,Alternative,40,0.011418783899514702
Easy Listening,74,951.3899999999994,0.015556022703384485,Easy Listening,24,0.006851270339708821
Electronica/Dance,55,614.7899999999997,0.01156190876602901,Electronica/Dance,30,0.008564087924636025


We could see that the range of tracks in the store (which is to say the quantity of tracks in the store) is a reflection from its total sales popularity. I included the sales column to show the sum total sales of each track with respect to the quantity sold.

We could see that the more the tracks in the store, the more quantity is sold and the more revenue is genrated. This is otherwise for Drama which has 64 tracks in the store but only recorded one sales.

Unlike Rock with total of 2635 tracks sold and at about 1297 available tracks in the store, with at about 37% of sales.

Also, we have some available tracks in the store which has not be sold and they include;
Bossa Nova,
Comedy,
Sci Fi & Fantasy etc


### Protected vs Non-protected media-types

In [94]:
%%sql --let us refresh our mind on how the media table look like

SELECT * FROM media_type
LIMIT 3

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file


In [103]:
%%sql

SELECT
    mt.name AS media_type_name,
    t.media_type_id,
    t.track_id,
    il.invoice_id,
    COUNT(il.quantity) AS total_quantity,
    SUM(i.total) AS total_sales,
    CAST(COUNT(il.quantity) AS FLOAT) / (SELECT )
FROM media_type AS mt
LEFT JOIN track AS t ON t.media_type_id = mt.media_type_id
LEFT JOIN invoice_line AS il ON il.track_id = t.track_id
LEFT JOIN invoice AS i ON i.invoice_id = il.invoice_id
GROUP BY 2
ORDER BY 6 DESC
    


Done.


media_type_name,media_type_id,track_id,invoice_id,total_quantity,total_sales
MPEG audio file,1,3335,320,4259,42934.32000000023
Protected AAC audio file,2,3503,None,439,4115.430000000001
Purchased AAC audio file,4,3498,None,35,274.2299999999999
AAC audio file,5,3359,360,21,153.44999999999996
Protected MPEG-4 video file,3,3429,None,3,25.740000000000002
